<a href="https://colab.research.google.com/github/fnovoas/sinrepco/blob/main/Sinrepco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SINREPCO
por fnovoas@unal.edu.co

1. Instalamos las bibliotecas necesarias: TensorFlow para el desarrollo de modelos de inteligencia artificial, OpenCV para el procesamiento de imágenes y Tesseract para el reconocimiento óptico de caracteres (OCR).

In [1]:
!pip install tensorflow opencv-python pytesseract

2. Procedemos con la recolección de datos, reunimos un conjunto de datos que contiene imágenes de vehículos, tanto contaminantes como no contaminantes, estas imágenes fueron capturadas por mí en diversas ubicaciones. Contamos con 3381 imágenes de vehículos no chimenea y 280 imágenes de vehículos chimenea.
Separamos las imágenes según si contienen vehículos que emiten humo visible o no, en dos carpetas: "chimenea" y "no_chimenea". Con estos datos entrenaremos al modelo.\
Montamos Drive para acceder a las imágenes.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/sinrepco_fotos"

Mounted at /content/drive
chimenea  no_chimenea


In [3]:
#importamos las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


3. Cargamos y preparamos los datos de la carpeta en donde están almacenados en Drive. Este código divide las imágenes en conjuntos de entrenamiento, validación y prueba según el porcentaje especificado: el 70% de los datos irán en entrenamiento y el 15% en cada una de las otras dos categorías. Ejecutar este código cambia la estructura de la organización de los archivos y carpetas en Drive.

In [4]:
import os
import random
import shutil
'''
#listar los elementos de la carpeta, para comprobar que accedió
import os
print(os.listdir(base_dir))
'''
def split_data(SOURCE, TRAINING, VALIDATION, TEST, SPLIT_SIZE):
    all_files = []
    for file_name in os.listdir(SOURCE):
        file_path = os.path.join(SOURCE, file_name)
        if os.path.getsize(file_path):
            all_files.append(file_name)
        else:
            print(f'{file_name} is zero length, so ignoring.')

    random.shuffle(all_files)
    train_size = int(len(all_files) * SPLIT_SIZE)
    val_size = int(len(all_files) * (1 - SPLIT_SIZE) / 2)

    train_files = all_files[:train_size]
    val_files = all_files[train_size:train_size + val_size]
    test_files = all_files[train_size + val_size:]

    for file_name in train_files:
        source_file = os.path.join(SOURCE, file_name)
        dest_file = os.path.join(TRAINING, file_name)
        shutil.copyfile(source_file, dest_file)

    for file_name in val_files:
        source_file = os.path.join(SOURCE, file_name)
        dest_file = os.path.join(VALIDATION, file_name)
        shutil.copyfile(source_file, dest_file)

    for file_name in test_files:
        source_file = os.path.join(SOURCE, file_name)
        dest_file = os.path.join(TEST, file_name)
        shutil.copyfile(source_file, dest_file)

# Configurar las rutas
base_dir = '/content/drive/My Drive/sinrepco_fotos'
source_chimenea = f'{base_dir}/chimenea'
source_no_chimenea = f'{base_dir}/no_chimenea'

train_chimenea_dir = f'{base_dir}/train/chimenea'
val_chimenea_dir = f'{base_dir}/validation/chimenea'
test_chimenea_dir = f'{base_dir}/test/chimenea'

train_no_chimenea_dir = f'{base_dir}/train/no_chimenea'
val_no_chimenea_dir = f'{base_dir}/validation/no_chimenea'
test_no_chimenea_dir = f'{base_dir}/test/no_chimenea'

os.makedirs(train_chimenea_dir, exist_ok=True)
os.makedirs(val_chimenea_dir, exist_ok=True)
os.makedirs(test_chimenea_dir, exist_ok=True)

os.makedirs(train_no_chimenea_dir, exist_ok=True)
os.makedirs(val_no_chimenea_dir, exist_ok=True)
os.makedirs(test_no_chimenea_dir, exist_ok=True)

# Dividir los datos
split_size = 0.7
split_data(source_chimenea, train_chimenea_dir, val_chimenea_dir, test_chimenea_dir, split_size)
split_data(source_no_chimenea, train_no_chimenea_dir, val_no_chimenea_dir, test_no_chimenea_dir, split_size)

Después de dividir los datos, configuramos el ImageDataGenerator para cargar las imágenes de estas nuevas carpetas:

In [5]:
# Parámetros
IMG_HEIGHT = 480
IMG_WIDTH = 640
BATCH_SIZE = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    f'{base_dir}/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    f'{base_dir}/validation',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    f'{base_dir}/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2562 images belonging to 2 classes.
Found 549 images belonging to 2 classes.
Found 550 images belonging to 2 classes.
